*Copyright 2019 StarkWare Industries Ltd.<br> Licensed under the Apache License, Version 2.0 (the "License"). You may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.starkware.co/open-source-license/ <br> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.*

# Part 3: FRI Commitments

- [Video Lecture (youtube)](https://www.youtube.com/watch?v=gd1NbKUOJwA)
- [Slides (PDF)](https://starkware.co/wp-content/uploads/2021/12/STARK101-Part3.pdf)

### Load Previous Session
Run the next cell to load the relevant variables. As usual - it will take a while to run.

In [1]:
from channel import Channel
from field import FieldElement
from merkle import MerkleTree
from polynomial import interpolate_poly, Polynomial
from tutorial_sessions import part1, part2

cp, cp_eval, cp_merkle, channel, eval_domain = part2()
print("Success")

100%|██████████| 1023/1023 [00:05<00:00, 177.79it/s]


Success


In [2]:
from field import FieldElement
from polynomial import interpolate_poly, X
from merkle import MerkleTree
from channel import Channel

"""
==================================================================
==============================Part.1==============================
==================================================================
"""

# 1. 使用群元素构建数列
a = [FieldElement(1), FieldElement(3141592)]
while len(a) < 1023:
    tmp = a[-1] * a[-1] + a[-2] * a[-2]
    a.append(tmp)

# 2. 使用生成元素构建数列
g = FieldElement.generator() ** (3 * 2**30 / 1024)
G = []
for i in range(1024):
    G.append(g**i)

# 3. 构建多项式
f = interpolate_poly(G[:-1], a)

# 4. 在Coset上进行评估
w = FieldElement.generator()
h = w ** ((3 * (2**30)) // 8192)
H = [h**i for i in range(8192)]
eval_domain = [w * x for x in H]
f_eval = [f(d) for d in eval_domain]

# 5. 构建Merkle树，构建commitment
f_merkle = MerkleTree(f_eval)

# 6. 使用Channel发送根节点
channel = Channel()
channel.send(f_merkle.root)


"""
==================================================================
==============================Part.2==============================
==================================================================
"""

# 1. 第一个约束
numer0 = f - 1
denom0 = X - 1
p0 = numer0 / denom0

# 2. 第二个约束
numer1 = f - 2338775057
denom1 = X - g**1022
p1 = numer1 / denom1

# 3. 第三个约束
# Construct a list `lst` of the linear terms (x-g**i):

numer2 = f(g**2 * X) - f(g * X) ** 2 - f**2
denom2 = (X**1024 - 1) / ((X - g**1021) * (X - g**1022) * (X - g**1023))
p2 = numer2 / denom2

print("deg p0 =", p0.degree())
print("deg p1 =", p1.degree())
print("deg p2 =", p2.degree())


# 4. 组合多项式
def get_CP(channel):
    alpha0 = channel.receive_random_field_element()
    alpha1 = channel.receive_random_field_element()
    alpha2 = channel.receive_random_field_element()
    return alpha0 * p0 + alpha1 * p1 + alpha2 * p2


# 5. 对组合多项式进行承诺
def CP_eval(channel):
    CP = get_CP(channel)
    result = []
    for d in eval_domain:
        result.append(CP(d))
    return result


# 实际上就是用Merkle树对CP进行承诺
# CP又是p0, p1, p2的组合多项式，把CP在陪集的定义域上进行赋值
channel = Channel()
CP_merkle = MerkleTree(CP_eval(channel))
channel.send(CP_merkle.root)

100%|██████████| 1023/1023 [00:05<00:00, 176.62it/s]


deg p0 = 1021
deg p1 = 1021
deg p2 = 1023


## FRI Folding

Our goal in this part is to construct the FRI layers and commit on them. 
<br>To obtain each layer we need:
1. To generate a domain for the layer (from the previous layer's domain).
2. To generate a polynomial for the layer (from the previous layer's polynomial and domain).
3. To evaluate said polynomial on said domain - **this is the next FRI layer**.

FRI Folding 是指通过逐层生成并处理多项式和域来构建 FRI 证明的过程。每一层的构建都基于前一层的域和多项式。具体来说，构建每一层需要完成以下三步：

1.	生成域（Domain）：
	
•	首先，从前一层的域中生成当前层的域。这通常涉及到对前一层域进行某种形式的折叠或缩减，使得新生成的域可以用于更低阶的多项式。

•	这个过程类似于将原有的验证问题“压缩”到一个更小、更简单的域上，从而简化接下来的验证工作。


2.	生成多项式（Polynomial）：

•	基于前一层的多项式和新的域，生成当前层的多项式。这个新多项式通常是通过对前一层多项式在新域上的评估或某种操作得到的。

•	这个步骤确保我们在每一层都能找到与前一层相关联的多项式，同时逐渐降低多项式的阶数，从而简化整个验证过程。


3.	在新域上评估多项式（Evaluate Polynomial）：

•	在新的域上评估生成的多项式，并将其结果作为新层的输出。这一层的输出将成为下一层的输入，逐层递归下去。

•	通过这种递归评估，最终可以验证原始问题是否满足要求。


### Domain Generation

The first FRI domain is simply the `eval_domain` that you already generated in Part 1, namely a coset of a group of order 8192. Each subsequent FRI domain is obtained by taking the first half of the previous FRI domain (dropping the second half), and squaring each of its elements.<br>

Formally - we got `eval_domain` by taking:<br>
$$w, w\cdot h, w\cdot h^2, ..., w\cdot h^{8191}$$

The next layer will therefore be:<br>
$$w^2, (w\cdot h)^2, (w\cdot h^2)^2, ..., (w\cdot h^{4095})^2$$

Note that taking the squares of the second half of each elements in `eval_domain` yields exactly
the same result as taking the squares of the first half. This is true for the next layers as well.

对当前域的后半部分元素进行平方操作，与对前半部分元素进行平方操作会产生相同的结果。

For example:

In [6]:
# eval_domain这些在part1和2里面已经做完了

print(eval_domain[100] ** 2)
half_domain_size = len(eval_domain) // 2
print(half_domain_size)
print(eval_domain[half_domain_size + 100] ** 2)

-373161870
4096
-373161870


Similarly, the domain of the third layer will be:<br>
$$w^4, (w\cdot h)^4, (w\cdot h^2)^4, ..., (w\cdot h^{2047})^4$$

And so on.

Write a function `next_fri_domain` that takes the previous domain as an argument, and outputs the next one.

In [9]:
def next_fri_domain(fri_domain):
    tmp=[]
    # 获取fri_domain的前一半的元素，再平方
    for i in range(len(fri_domain)//2): 
        tmp.append(fri_domain[i]**2)
    return tmp

Solution:

In [4]:
def next_fri_domain(fri_domain):
    return [x ** 2 for x in fri_domain[:len(fri_domain) // 2]]

Run test: 

In [8]:
# Test against a precomputed hash.
from hashlib import sha256
next_domain = next_fri_domain(eval_domain)
assert '5446c90d6ed23ea961513d4ae38fc6585f6614a3d392cb087e837754bfd32797' == sha256(','.join([str(i) for i in next_domain]).encode()).hexdigest()
print('Success!')

Success!


### FRI Folding Operator
The first FRI polynomial is simply the composition polynomial, i.e., `cp`.<br>
Each subsequent FRI polynomial is obtained by:
1. Getting a random field element $\beta$ (by calling `Channel.receive_random_field_element`).
2. Multiplying the odd coefficients of the previous polynomial by $\beta$.
3. Summing together consecutive pairs (even-odd) of coefficients.

Formally, let's say that the k-th polynomial is of degree $< m$ (for some $m$ which is a power of 2):

$$p_{k}(x) := \sum _{i=0} ^{m-1} c_i x^i$$


Then the (k+1)-th polynomial, whose degree is $< \frac m 2 $ will be:

$$p_{k+1}(x) := \sum _{i=0} ^{  m / 2 - 1 } (c_{2i} + \beta \cdot c_{2i + 1}) x^i$$

这里理解一下就是在做逐渐的降低阶的过程：

先选取一个随机数

然后用这个随机数乘上一层多项式度奇数系数

再加上偶数系数，就成为了新的各个项的系数

这样x的项变少了一半 实现了阶的降低



Write a function `next_fri_polynomial` that takes as arguments a polynomial and a field element (the one we referred to as $\beta$), and returns the "folded" next polynomial.

Note that:
1. `Polynomial.poly` contains a list of a polynomial's coefficients, the free term first, and the highest degree last, so `p.poly[i] == u` if the coefficient of $x^i$ is $u$.*
2. `Polynomial`'s default constructor takes the list of coefficients as argument. So a polynomial can be instantiated from a list of coefficients `l` by calling `Polynomial(l)`.


In [25]:
def next_fri_polynomial(poly, beta):
    # 这里取出来了两个系数列表，一个是奇数项，一个是偶数项
    odd_coefficients = poly.poly[1::2] # No need to fix this line.
    even_coefficients = poly.poly[::2] # No need to fix this line either.
    # 系数列表减半了，所以多项式的阶也就减半了，实现了多项式的降阶
    # 然后在这里构造了新的多项式
    odd = beta * Polynomial(odd_coefficients)
    even = Polynomial(even_coefficients)
    return odd+even

Solution:

In [24]:
def next_fri_polynomial(poly,  beta):
    odd_coefficients = poly.poly[1::2]
    even_coefficients = poly.poly[::2]
    odd = beta * Polynomial(odd_coefficients)
    even = Polynomial(even_coefficients)
    return odd + even

Run test:

In [15]:
next_p = next_fri_polynomial(cp, FieldElement(987654321))
assert '6bff4c35e1aa9693f9ceb1599b6a484d7636612be65990e726e52a32452c2154' == sha256(','.join([str(i) for i in next_p.poly]).encode()).hexdigest()
print('Success!')

511
Success!


### Putting it Together to Get the Next FRI Layer

Write a function `next_fri_layer` that takes a polynomial, a domain, and a field element (again - $\beta$), and returns the next polynomial, the next domain, and the evaluation of this next polynomial on this next domain.

In [18]:
def next_fri_layer(poly, domain, beta):
    next_poly = next_fri_polynomial(poly, beta) 
    next_domain = next_fri_domain(domain)
    next_layer = []
    for x in next_domain:
        next_layer.append(next_poly(x))
    return next_poly, next_domain, next_layer

Solution:

In [17]:
def next_fri_layer(poly, domain, beta):
    next_poly = next_fri_polynomial(poly, beta)
    next_domain = next_fri_domain(domain)
    next_layer = [next_poly(x) for x in next_domain]
    return next_poly, next_domain, next_layer

Run test:

In [26]:
test_poly = Polynomial([FieldElement(2), FieldElement(3), FieldElement(0), FieldElement(1)])
test_domain = [FieldElement(3), FieldElement(5)]
beta = FieldElement(7)
next_p, next_d, next_l = next_fri_layer(test_poly, test_domain, beta)
assert next_p.poly == [FieldElement(23), FieldElement(7)]
assert next_d == [FieldElement(9)]
assert next_l == [FieldElement(86)]
print('Success!')

Success!


## Generating FRI Commitments

We have now developed the tools to write the `FriCommit` method, that contains the main FRI commitment loop.<br>

It takes the following 5 arguments:
1. The composition polynomial, that is also the first FRI polynomial, that is - `cp`.
2. The coset of order 8192 that is also the first FRI domain, that is - `eval_domain`.
3. The evaluation of the former over the latter, which is also the first FRI layer , that is - `cp_eval`.
4. The first Merkle tree (we will have one for each FRI layer) constructed from these evaluations, that is - `cp_merkle`.
5. A channel object, that is `channel`.

The method accordingly returns 4 lists:
1. The FRI polynomials.
2. The FRI domains.
3. The FRI layers.
4. The FRI Merkle trees.

The method contains a loop, in each iteration of which we extend these four lists, using the last element in each.
The iteration should stop once the last FRI polynomial is of degree 0, that is - when the last FRI polynomial is just a constant. It should then send over the channel this constant (i.e. - the polynomial's free term).
The `Channel` class only supports sending strings, so make sure you convert anything you wish to send over the channel to a string before sending.

In [27]:
# Fix this according to the instructions (lines with no specific comments are okay).
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while fri_polys[-1].degree()>0: # Replace this with the correct halting condition.
        beta = channel.receive_random_field_element() # Change to obtain a random element from the channel.
        next_poly, next_domain, next_layer = next_fri_layer(fri_polys[-1],fri_domains[-1],beta) # Fix to obtain the next FRI polynomial, domain, and layer.
        
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        
        fri_merkle=MerkleTree(next_layer)
        channel.send(fri_merkle.root) # Fix to send the correct commitment.
        fri_merkles.append(fri_merkle) # Fix to construct the correct Merkle tree.
    channel.send(str(fri_polys[-1].poly[0])) # Fix to send the last layer's free term.
    return fri_polys, fri_domains, fri_layers, fri_merkles

Solution:

In [28]:
def FriCommit(cp, domain, cp_eval, cp_merkle, channel):    
    fri_polys = [cp]
    fri_domains = [domain]
    fri_layers = [cp_eval]
    fri_merkles = [cp_merkle]
    while fri_polys[-1].degree() > 0:
        beta = channel.receive_random_field_element()
        next_poly, next_domain, next_layer = next_fri_layer(fri_polys[-1], fri_domains[-1], beta)
        fri_polys.append(next_poly)
        fri_domains.append(next_domain)
        fri_layers.append(next_layer)
        fri_merkles.append(MerkleTree(next_layer))
        channel.send(fri_merkles[-1].root)   
    channel.send(str(fri_polys[-1].poly[0]))
    return fri_polys, fri_domains, fri_layers, fri_merkles

Run test:

In [29]:
test_channel = Channel()
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, test_channel)
assert len(fri_layers) == 11, f'Expected number of FRI layers is 11, whereas it is actually {len(fri_layers)}.'
assert len(fri_layers[-1]) == 8, f'Expected last layer to contain exactly 8 elements, it contains {len(fri_layers[-1])}.'
assert all([x == FieldElement(-1138734538) for x in fri_layers[-1]]), f'Expected last layer to be constant.'
assert fri_polys[-1].degree() == 0, 'Expacted last polynomial to be constant (degree 0).'
assert fri_merkles[-1].root == '1c033312a4df82248bda518b319479c22ea87bd6e15a150db400eeff653ee2ee', 'Last layer Merkle root is wrong.'
assert test_channel.state == '61452c72d8f4279b86fa49e9fb0fdef0246b396a4230a2bfb24e2d5d6bf79c2e', 'The channel state is not as expected.'
print('Success!')

Success!


Run the following cell to execute the function with your channel object and print the proof so far:

In [15]:
fri_polys, fri_domains, fri_layers, fri_merkles = FriCommit(cp, eval_domain, cp_eval, cp_merkle, channel)
print(channel.proof) 

['send:6c266a104eeaceae93c14ad799ce595ec8c2764359d7ad1b4b7c57a4da52be04', 'receive_random_field_element:2948900820', 'receive_random_field_element:1859037345', 'receive_random_field_element:2654806830', 'send:61f7d8283e244d391a483c420776e351fcfdbce525a698461a8307a1345b5652', 'receive_random_field_element:394024765', 'send:9431516ee735a498c4aec3da30112e417b03e55e5be939ff44ca8a0a62475b15', 'receive_random_field_element:1705983878', 'send:584b4b88a7f296efa0309d8e6faef13573b1ee5dfcb02ed8be5d853172f3fc69', 'receive_random_field_element:665918954', 'send:2debb983bb6473a5d4e9046944fb7ef66ef814c64f58ca5d8ebc2a15ed61ca4a', 'receive_random_field_element:3182659911', 'send:5da75aa9d9a9a564d7f19e431cbbb91eff030c353f3825dc5352674d1b7813f9', 'receive_random_field_element:2692084106', 'send:8ca6b618f3d758e7a99c4988e3a30e5c443f6f4ed79c64b698b031cca67ee4c2', 'receive_random_field_element:2453626065', 'send:db00ee380f0b1a9c2aa37efe2eabca468a83370046cf824eea9409e536256996', 'receive_random_field_element: